# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [120]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [121]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [122]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [123]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [124]:
def foldl_with_foldr(f,x0,lst):
    composed=foldr(lambda seqval,acc: lambda n: acc(f(n, seqval)),lambda identity: identity,lst)
    return composed(x0)

In [125]:
def foldr_with_foldl(f,x0,lst):
    composed=foldl(lambda acc,seqval: lambda n: acc(f(seqval,n)),lambda identity: identity,lst)
    return composed(x0)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [ ]:
#n - len of a; m - len of b
def check_inv(a, b):  
    b_dict = {ch : 0 for ch in b} #O(m)
    b_dict_check = dict() #O(1)
    for ch in b: #O(m)
        b_dict[ch] = b_dict[ch] + 1 #O(1)
    for i in range(0, len(a)): #O(n)
        is_added = False
        if a[i] in b_dict: #O(1)
            is_added = True
            if a[i] in b_dict_check: #O(1)
                b_dict_check[a[i]] = b_dict_check[a[i]] + 1 #O(1)
            else:
                b_dict_check[a[i]] = 1 #O(1)
        else:
            b_dict_check.clear() #O(1)
            is_added = False
        if is_added:
            if b_dict == b_dict_check:
                return True
    return False

In [58]:
check_inv('rtmo','rto')

False

In [61]:
check_inv('abcrotm','rot')

True

## In worst O(k^2), in average O(n*k)

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [14]:

class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left:
            yield from iter(self.left)
        if self.right:
            yield from iter(self.right)
        else:
            yield self.value
    
    def __str__(self):
        return '{val}'.format(val = self.value)
    
    def __repr__(self):
        return 'Tree({val}, {l}, {r})'.format(val = self.value, 
                                              l = repr(self.left), 
                                              r = repr(self.right))
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

print(list(tree) == [3, 4, 2])
print(list(tree))
print(tree.__repr__())

True
[3, 4, 2]
Tree(0, Tree(1, Tree(3, None, None), Tree(4, None, None)), Tree(2, None, None))


# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

I decided to use simplest solution of problem - Reverse Polish Notation:

In [62]:
import re
def getPriority(symbol):
    if(symbol == '*' or symbol == '/'):
        return 3;
    if(symbol == '+' or symbol == '-'):
        return 2;  
    if(symbol == "("):
        return 1
    if (symbol == ")"):
        return -1
    return 0; 

Method to handle cases such -2 and (-2):

In [63]:
def hadle_special_cases(expression):
    preparedExpression=''
    for i in range(len(expression)):
        currentSymbol = expression[i]
        
        if(currentSymbol == '-'):
            if(i == 0):
                preparedExpression+='0'
                
            if(expression[i-1] == '('):
                preparedExpression+='0'
            
        preparedExpression+=currentSymbol
        
    return preparedExpression

Parse input string to RPN:

In [64]:
def parse_expression_to_RPN(expression):
    expression = hadle_special_cases(expression)
    rpnExpression=''
    operationStack=[]
    
    for i in range (len(expression)):
        currentSymbol = expression[i]
        currentPriority=getPriority(expression[i])
        
        if(currentPriority == 0):
            rpnExpression+=currentSymbol
            
        if(currentPriority == 1):
            operationStack.append(currentSymbol)
            
        if (currentPriority >1):
            rpnExpression+=' '
            while (operationStack):
                if (getPriority(operationStack[-1])>=currentPriority):
                    rpnExpression+=operationStack.pop()
                    rpnExpression+=' '
                else: break
            operationStack.append(currentSymbol)
                    
        if (currentPriority == -1):
            rpnExpression+=' '
            while (getPriority(operationStack[-1]) != 1):
                 rpnExpression+=operationStack.pop()
                 rpnExpression+=' '
            operationStack.pop()
       
    while (operationStack):
        rpnExpression+=' '
        rpnExpression+=operationStack.pop();
    return rpnExpression
    

parse fron RPN to digit answer:

In [65]:
operations = {
  "+": (lambda a, b: a + b),
  "-": (lambda a, b: a - b),
  "*": (lambda a, b: a * b),
  "/": (lambda a, b: a / b)
}

In [66]:
def parse_RPN_to_answer(rpn):
    tokens = rpn.split()
    resultStack= []

    for token in tokens:
        if token in operations:
            b = resultStack.pop()
            a = resultStack.pop()
            result = operations[token](a, b)
            resultStack.append(result)
        else:
            resultStack.append(int(token))

    return resultStack.pop()

In [67]:
def calc(expr):
    rpn=parse_expression_to_RPN(expr)
    return parse_RPN_to_answer(rpn)

calc('2 * (15 - 3 * 4) - 2')==4

True